In [2]:
import xml.etree.ElementTree as ET
import os

def convert_directory_xml_to_yolo(directory, classes_dict):
    """
    Convert all XML files in a directory to YOLO format .txt files.

    Args:
    - directory (str): Path to the directory containing XML files.
    - classes_dict (dict): A dictionary mapping class names to class IDs.
    """

    for filename in os.listdir(directory):
        if filename.endswith('.xml'):
            xml_file_path = os.path.join(directory, filename)
            tree = ET.parse(xml_file_path)
            root = tree.getroot()

            size = root.find('size')
            width = int(size.find('width').text)
            height = int(size.find('height').text)

            yolo_data = []

            for member in root.findall('object'):
                class_name = member.find('name').text
                if class_name in classes_dict:
                    class_id = classes_dict[class_name]
                else:
                    continue

                bndbox = member.find('bndbox')
                xmin = int(bndbox.find('xmin').text)
                ymin = int(bndbox.find('ymin').text)
                xmax = int(bndbox.find('xmax').text)
                ymax = int(bndbox.find('ymax').text)

                x_center = round(((xmin + xmax) / 2) / width, 3)
                y_center = round(((ymin + ymax) / 2) / height, 3)
                obj_width = round((xmax - xmin) / width, 3)
                obj_height = round((ymax - ymin) / height, 3)


                yolo_data.append(f"{class_id} {x_center} {y_center} {obj_width} {obj_height}")

            output_file_path = os.path.join(directory, filename.replace('.xml', '.txt'))
            with open(output_file_path, 'w') as file:
                file.write('\n'.join(yolo_data))

# Usage example
classes_dict = {
    "Anomaly": 0
}

for split in ['train', 'valid']:
    directory = f'./detection_single/labels/{split}'
    convert_directory_xml_to_yolo(directory, classes_dict)

In [3]:
import os
import shutil

In [4]:
def move_xml_files(source_dir, target_dir):
    """
    Move all .xml files from source_dir to target_dir.

    Args:
    source_dir (str): The directory to move files from.
    target_dir (str): The directory to move files to.

    Returns:
    None
    """
    # Check if the target directory exists, if not create it
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # Move each .xml file from source to target directory
    for file_name in os.listdir(source_dir):
        if file_name.endswith('.xml'):
            shutil.move(os.path.join(source_dir, file_name), target_dir)

In [6]:
move_xml_files("./detection_single/labels/valid", "./detection_single/labels_xml/valid")

In [8]:
for k in range(1, 6):
    for split in ['train', 'valid']:
        directory = f'./detection_multi_folds/Kfold{k}/labels/{split}'
        target_dir = f'./detection_multi_folds/Kfold{k}/labels_xml/{split}'
        move_xml_files(directory, target_dir)